# LangChain `@tool` 데코레이터

In [223]:
from datetime import datetime
from dotenv import load_dotenv
import pytz
from langchain_core.tools import tool   # decorator
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [224]:
# .env 파일에 저장된 api_key를 OS 환경 변수로 로딩
load_dotenv()

True

In [225]:
# OpenAI 클라이언트를 생성
model = ChatOpenAI(model = 'gpt-4o-mini')

In [226]:
# AI에게 메시지 전달하고 실행(invoke).
messages = [
    SystemMessage(content='너는 사용자의 질문에 답하는 AI 비서야.'),
    HumanMessage(content='지금 현재 서울 시간?')
]
ai_message = model.invoke(input=messages)

In [227]:
type(ai_message)

langchain_core.messages.ai.AIMessage

In [228]:
print(ai_message)

content='현재 서울 시간은 알 수 없지만, 서울은 한국 표준시(KST)로 UTC+9에 해당합니다. 현재 시간은 사용자의 기기에서 확인하실 수 있습니다. 기기의 시간대 설정이 올바른지 확인해 주세요.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 30, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CGK0U4wRfkRqmCg7AbZJa0Uj2dge6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--5c705c3d-80b7-46dd-9730-3c0cd641f31f-0' usage_metadata={'input_tokens': 30, 'output_tokens': 54, 'total_tokens': 84, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [229]:
ai_message.pretty_print()

================================== Ai Message ==================================

현재 서울 시간은 알 수 없지만, 서울은 한국 표준시(KST)로 UTC+9에 해당합니다. 현재 시간은 사용자의 기기에서 확인하실 수 있습니다. 기기의 시간대 설정이 올바른지 확인해 주세요.


도구(tool)을 AI에게 제공하고, AI는 tool 목록에 있는 함수 호출(function calling)을 요청해서 에이전트가 함수 호출 결과를 다시 AI에게 전송하면 AI는 함수 호출 결과를 바탕으로 답변을 생성할 수 있음.

사용자 질문(도구 목록 제공) -> AI 도구 호출 요청 -> 사용자 함수 호출 결과 제공 -> AI 답변 생성

In [230]:
@tool
def get_current_time(timezone: str, location: str) -> str:
    """해당 timezone의 현재 날짜와 시간을 문자열로 리턴.

    Args:
        timezone (str) : 타임존. (예: Asia/Seoul)
        location (str) : 지역명. 타임존은 모든 도시 이름에 대응되지 않기 때문에 LLM이 답변을 생성할 때 이용하도록 제공.
    Returns: '날짜 시간 타임존(지역명)' 형식의 문자열을 반환. (예: 2025-09-15 15:30:55 Asia/Seoul(부산))
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')
    result = f'{now} {timezone}({location})'

    return result

In [231]:
# 도구 목록 : @tool 데코레이터가 사용된 함수들의 리스트
tools = [get_current_time,]

In [232]:
# AI에서 도구 목록의 함수 호출을 요청했을 때 함수 객체를 쉽게 찾기 위해서 dict 선언.
tool_dict = {
    'get_current_time': get_current_time,
}

In [233]:
# AI 모델과 도구 목록을 binding (묶어줌).
model_with_tools = model.bind_tools(tools=tools)

In [234]:
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답하는 AI 비서야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='지금 현재 서울 시간?', additional_kwargs={}, response_metadata={})]


In [235]:
# 도구 목록을 가지고 있는 AI 모델 호출
response = model_with_tools.invoke(input=messages)

In [236]:
print(type(response))

<class 'langchain_core.messages.ai.AIMessage'>


In [237]:
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_CRJM7ekTQfM0L1A5SRmolVCY', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"서울"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 170, 'total_tokens': 192, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CGK0WlmNFQ6pD4I73Z5Sqj8EwatzH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--e3602f5d-1694-41b7-be70-7fc455645545-0' tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '서울'}, 'id': 'call_CRJM7ekTQfM0L1A5SRmolVCY', 'type': 'tool_call'}] usage_metadata={'input_tokens': 170, 'output_tokens': 22, '

In [238]:
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_CRJM7ekTQfM0L1A5SRmolVCY)
 Call ID: call_CRJM7ekTQfM0L1A5SRmolVCY
  Args:
    timezone: Asia/Seoul
    location: 서울


In [239]:
messages.append(response)   # 대화 이력 저장 : AIMessages 객체를 리스트에 추가

In [240]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

너는 사용자의 질문에 답하는 AI 비서야.
================================ Human Message =================================

지금 현재 서울 시간?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_CRJM7ekTQfM0L1A5SRmolVCY)
 Call ID: call_CRJM7ekTQfM0L1A5SRmolVCY
  Args:
    timezone: Asia/Seoul
    location: 서울


In [241]:
for tool_call in response.tool_calls:
    fn = tool_dict[tool_call['name']]   # AIMessage의 포함된 함수 이름으로 함수 객체를 찾음.
    tool_msg = fn.invoke(tool_call)     # @tool 데코레이터로 포장 -> invoke가 가능.
    # invoke : tool_call에서 args를 찾고 fn를 호출할 때 아규먼트를 전달. 리턴값을 이용해서 ToolMessage 객체 반환.
    messages.append(tool_msg)

In [242]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

너는 사용자의 질문에 답하는 AI 비서야.
================================ Human Message =================================

지금 현재 서울 시간?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_CRJM7ekTQfM0L1A5SRmolVCY)
 Call ID: call_CRJM7ekTQfM0L1A5SRmolVCY
  Args:
    timezone: Asia/Seoul
    location: 서울
================================= Tool Message =================================
Name: get_current_time

2025-09-16 16:03:12 Asia/Seoul(서울)


In [243]:
ai_message = model_with_tools.invoke(input=messages)

In [244]:
ai_message.pretty_print()

================================== Ai Message ==================================

현재 서울의 시간은 2025년 9월 16일 16:03:12입니다.


# 2개 이상의 도구 호출을 포함하는 tool_calls 리스트

In [245]:
messages = [
    SystemMessage('사용자의 질문에 답하기 위해서 도구를 사용할 수 있어.'),
    SystemMessage('도시 이름과 타임존은 일치하지 않을 수도 있으니, 정확한 타임존을 선택해줘.'),
    HumanMessage('서울, LA, 런던의 현재 시간은?'),
]

In [246]:
ai_message = model_with_tools.invoke(input=messages)

In [247]:
messages.append(ai_message)

In [248]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

사용자의 질문에 답하기 위해서 도구를 사용할 수 있어.
================================ System Message ================================

도시 이름과 타임존은 일치하지 않을 수도 있으니, 정확한 타임존을 선택해줘.
================================ Human Message =================================

서울, LA, 런던의 현재 시간은?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_egUhzj0n9SOdhhL9WXX1lL2p)
 Call ID: call_egUhzj0n9SOdhhL9WXX1lL2p
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_JXALW9OJznhGbd7iWZdTE7N3)
 Call ID: call_JXALW9OJznhGbd7iWZdTE7N3
  Args:
    timezone: America/Los_Angeles
    location: LA
  get_current_time (call_LI4Gi1dyvJMaUReQqPny1lLP)
 Call ID: call_LI4Gi1dyvJMaUReQqPny1lLP
  Args:
    timezone: Europe/London
    location: 런던


In [249]:
for tool_call in ai_message.tool_calls:
    fn = tool_dict[tool_call['name']]
    tool_msg = fn.invoke(tool_call)
    messages.append(tool_msg)

In [250]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

사용자의 질문에 답하기 위해서 도구를 사용할 수 있어.
================================ System Message ================================

도시 이름과 타임존은 일치하지 않을 수도 있으니, 정확한 타임존을 선택해줘.
================================ Human Message =================================

서울, LA, 런던의 현재 시간은?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_egUhzj0n9SOdhhL9WXX1lL2p)
 Call ID: call_egUhzj0n9SOdhhL9WXX1lL2p
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_JXALW9OJznhGbd7iWZdTE7N3)
 Call ID: call_JXALW9OJznhGbd7iWZdTE7N3
  Args:
    timezone: America/Los_Angeles
    location: LA
  get_current_time (call_LI4Gi1dyvJMaUReQqPny1lLP)
 Call ID: call_LI4Gi1dyvJMaUReQqPny1lLP
  Args:
    timezone: Europe/London
    location: 런던
================================= Tool Message =================================
Name: get_current_time

2025-09-16 16:03:16 Asia/Se

In [251]:
ai_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_egUhzj0n9SOdhhL9WXX1lL2p)
 Call ID: call_egUhzj0n9SOdhhL9WXX1lL2p
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_JXALW9OJznhGbd7iWZdTE7N3)
 Call ID: call_JXALW9OJznhGbd7iWZdTE7N3
  Args:
    timezone: America/Los_Angeles
    location: LA
  get_current_time (call_LI4Gi1dyvJMaUReQqPny1lLP)
 Call ID: call_LI4Gi1dyvJMaUReQqPny1lLP
  Args:
    timezone: Europe/London
    location: 런던


# Pydantic

입력된 데이터의 유효성과 형식을 검증하고 특정 데이터 형식으로 명확히 표현하는 라이브러리.

In [252]:
def subtract(x: int, y: int) -> int:
    return x - y

In [253]:
subtract(1, 2)

-1

In [254]:
subtract(1.1, 2.5)

-1.4

Python은 파라미터의 타입 검사를 하지 않고, 리턴 타입도 힌트대로 반환하는 것은 아님.

In [255]:
subtract('a', 'b')

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [290]:
class Person:
    def __init__(self, id: int, name: str):
        self.id = id
        self.name = name

In [291]:
person = Person(id=1, name='오쌤')

In [292]:
print(person.id)

1


In [293]:
person2 = Person(id='abc123', name=123)

In [294]:
print(person2.id)

abc123


Python은 객체를 생성할 때 속성(property)들의 타입 검사를 수행하지 않음.

In [295]:
from pydantic import BaseModel, ValidationError, PositiveInt

In [296]:
class Person(BaseModel):    # 부모 클래스 상속, Person 클래스는 BaseModel 클래스를 상속(확장).
    id: PositiveInt
    name: str

In [297]:
person3 = Person(id=12345, name='홍길동')

In [298]:
print(person3)

id=12345 name='홍길동'


In [299]:
person3.model_dump()    # 인스턴스의 속성(property) 이름과 값으로 dict를 생성해서 리턴.

{'id': 12345, 'name': '홍길동'}

In [300]:
try:
    person4 = Person(id='abc123', name='1_000')
    print(person4)
except ValidationError as e:
    print(e)

1 validation error for Person
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


In [301]:
person5 = Person(**{'id' : 123456, 'name' : '길동'})  # ** : unpacking 연산자
print(person5)

id=123456 name='길동'


# Pydandtic을 이용한 함수 파라미터 타입 클래스 선언

In [302]:
from pydantic import Field

In [303]:
# 주식 가격 조회 함수의 파라미터 타입으로 사용할 클래스.
class StockHistoryArgs(BaseModel):
    ticker: str = Field(..., title='주식 코드',
                        description='주식 데이터를 검색하기 위한 주식 코드(예: AAPL, AMZN)')
    period: str = Field(..., title='기간',
                        description='주식 데이터 조회 기간(예: 1d, 1mo, 1y)')

In [304]:
StockHistoryArgs.model_json_schema()

{'properties': {'ticker': {'description': '주식 데이터를 검색하기 위한 주식 코드(예: AAPL, AMZN)',
   'title': '주식 코드',
   'type': 'string'},
  'period': {'description': '주식 데이터 조회 기간(예: 1d, 1mo, 1y)',
   'title': '기간',
   'type': 'string'}},
 'required': ['ticker', 'period'],
 'title': 'StockHistoryArgs',
 'type': 'object'}

In [305]:
import yfinance as yf

In [306]:
@tool
def get_yf_stock_history(input: StockHistoryArgs):
    """해당 기간(period) 동안 주식 코드(ticker)의 데이터를 조회하는 함수. """
    ticker = yf.Ticker(input.ticker)
    history = ticker.history(period=input.period)
    return history.to_markdown()    # 조회한 데이터를 마크다운(Markdown) 형식의 문자열로 리턴.

In [307]:
tools = [get_current_time, get_yf_stock_history,]

In [308]:
tool_dict ={
    'get_current_time': get_current_time,
    'get_yf_stock_history': get_yf_stock_history,
}

In [309]:
model_with_tools = model.bind_tools(tools=tools)    # 도구 목록을 포함하는 AI 모델 객체

In [310]:
messages = [
    SystemMessage('답변을 생성하기 위해 도구를 사용할 수 있어.'),
    SystemMessage('도시 이름과 타임존은 일치하지 않을 수도 있어.'),
    SystemMessage('ticker는 실제 사용되는 주식 코드여야 해.'),
    HumanMessage('삼성전자는 한 달 전과 비교해서 주식이 올랐어? 내렸어?'),
]

In [311]:
ai_message = model_with_tools.invoke(input=messages)

In [312]:
messages.append(ai_message)

In [313]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

답변을 생성하기 위해 도구를 사용할 수 있어.
================================ System Message ================================

도시 이름과 타임존은 일치하지 않을 수도 있어.
================================ System Message ================================

ticker는 실제 사용되는 주식 코드여야 해.
================================ Human Message =================================

삼성전자는 한 달 전과 비교해서 주식이 올랐어? 내렸어?
================================== Ai Message ==================================
Tool Calls:
  get_yf_stock_history (call_ZbkrSu2IJrRVLLg6MHfZz8Xx)
 Call ID: call_ZbkrSu2IJrRVLLg6MHfZz8Xx
  Args:
    input: {'ticker': '005930.KS', 'period': '1mo'}


In [314]:
for tool_call in ai_message.tool_calls:
    fn = tool_dict[tool_call['name']]
    tool_msg = fn.invoke(tool_call)
    messages.append(tool_msg)

In [315]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

답변을 생성하기 위해 도구를 사용할 수 있어.
================================ System Message ================================

도시 이름과 타임존은 일치하지 않을 수도 있어.
================================ System Message ================================

ticker는 실제 사용되는 주식 코드여야 해.
================================ Human Message =================================

삼성전자는 한 달 전과 비교해서 주식이 올랐어? 내렸어?
================================== Ai Message ==================================
Tool Calls:
  get_yf_stock_history (call_ZbkrSu2IJrRVLLg6MHfZz8Xx)
 Call ID: call_ZbkrSu2IJrRVLLg6MHfZz8Xx
  Args:
    input: {'ticker': '005930.KS', 'period': '1mo'}
================================= Tool Message =================================
Name: get_yf_stock_history

| Date                      |   Open |   High |   Low |   Close |      Volume |   Dividends |   Stock Splits |
|:--------------------------|-------:|-------:|------:|--------:|------------:|------------:

In [316]:
ai_message = model_with_tools.invoke(input=messages)

In [317]:
ai_message.pretty_print()

================================== Ai Message ==================================

삼성전자의 주가는 한 달 전과 비교하여 다음과 같은 변화가 있었습니다.

- 한 달 전의 주가 (2025년 8월 18일): 70,000 원
- 현재 주가 (2025년 9월 16일): 79,400 원

따라서, 삼성전자는 한 달 동안 주가가 상승했습니다.
